# Import

In [ ]:
import pandas as pd

# Data Loading

In [1]:
def download_csv_from_google_drive(url):
    """
    Downloads a CSV file from a Google Drive link and returns it as a pandas DataFrame.

    Parameters:
    url (str): The link to the Google Drive file shared publicly.

    Returns:
    pandas.DataFrame: A DataFrame containing the data from the downloaded CSV file.
    """
    # Construct the Google Drive direct download link
    file_id = url.split('/')[-2]
    direct_download_url = 'https://drive.google.com/uc?id=' + file_id

    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(direct_download_url)

    return df

def download_file_from_google_drive(url):
    """
    Downloads a file from a publicly shared Google Drive link and returns it.

    Parameters:
    url (str): The link to the publicly shared Google Drive file.

    Returns:
    None
    """
    # Extract the file ID from the URL
    file_id = url.split('/')[-2]

    # Download the file using the gdown command
    !gdown $file_id

def search_and_download_images(categories, search_terms, num_images_per_category, base_dir):
    """
    Search and download images using DuckDuckGo search.

    Args:
        categories (list): List of category names.
        search_terms (list): List of search terms corresponding to each category.
        num_images_per_category (int): Number of images to download per category.
        base_dir (str): Base directory where images will be saved.

    Returns:
        None

    Example Usage:
    --------------
    categories = ["Grizzly Bear", "Black Bear", "Teddy Bear"]
    search_terms = ["grizzly bear", "black bear", "teddy bear"]
    num_images_per_category = 50
    base_dir = "bears_dataset"

    search_and_download_images(categories, search_terms, num_images_per_category, base_dir)
    """

    try:
        from duckduckgo_search import ddg_images
    except ImportError:
        !pip install -q fastai duckduckgo_search
        from duckduckgo_search import ddg_images

    from fastai.vision.utils import download_images
    from pathlib import Path
    from fastcore.all import L

    for category, search_term in zip(categories, search_terms):
        folder_name = category.replace(" ", "_").lower()
        dest = Path(base_dir) / folder_name
        dest.mkdir(parents=True, exist_ok=True)

        search_results = L(ddg_images(search_term, max_results=num_images_per_category)).itemgot('image')
        download_images(dest, urls=search_results)

        print(f"Downloaded {num_images_per_category} images for '{search_term}' and saved to '{dest}'.")


# Data Checking

In [2]:
def analyze_columns(df):
    """
    Analyzes each column in a DataFrame by displaying the column name, the number of unique values,
    and the count of each unique value in the column.

    Parameters:
    df (pandas.DataFrame): The DataFrame to be analyzed.
    """
    for column_name in df.columns:
        print(f'Column name: {column_name}, number of unique values: {df[column_name].nunique()}')
        print(df[column_name].value_counts().sort_index())
        print('-----------')
def analyze_duplicates_and_missing_values(df):
    """
    Analyzes a DataFrame for duplicated rows and missing values in columns.
    Displays the total count of duplicated rows and a summary of columns with missing values.

    Parameters:
    df (pandas.DataFrame): The DataFrame to be analyzed.
    """
    # Analyze duplicated rows
    duplicated_count = df.duplicated().sum()
    print(f'Total count of duplicated rows: {duplicated_count}')
    print('----')

    # Analyze missing values in columns
    missing_values = df.isna().sum()
    missing_values = missing_values[missing_values > 0]
    if not missing_values.empty:
        print(f'DataFrame length: {len(df)}\nColumns with missing values: \n{missing_values}')
    else:
        print('No columns with missing values found')



# Data Visualising

In [3]:
def correlation_matrix_with_phik(df, target=None, height=14, width=9):
    """
    Generates a correlation matrix using the Phik library and displays it as a heatmap.

    Parameters:
    df (pandas.DataFrame): The DataFrame for which to generate the correlation matrix.
    target (str): The name of the target column in the DataFrame (if specified).
    height (int): Height of the heatmap figure.
    width (int): Width of the heatmap figure.
    """
    # Check if phik is already installed
    try:
        import phik
    except ImportError:
        !pip -q install phik

    # Import necessary libraries
    from phik.report import plot_correlation_matrix
    import seaborn as sns
    import matplotlib.pyplot as plt

    # Calculate the Phik correlation matrix
    phik_overview = df.phik_matrix()

    # Create a figure with a heatmap
    plt.figure(figsize=(height, width))

    # Customize the heatmap appearance
    sns.heatmap(phik_overview, annot=True, fmt=".2f", annot_kws={"size": 8}, cmap="coolwarm")

    # If a target column is specified, display its correlation with other columns
    if target is not None:
        from phik import report
        phik_sorted = phik_overview[target].sort_values(ascending=False)
        print(f"\nCorrelation of '{target}' with other columns:")
        print(f'{phik_sorted}\n')


12. 200.9
13. Нельзя создать экземпляр абстрактного класса